# webcam

In [1]:
import cv2
import numpy as np

# Abrir o vídeo
cam = cv2.VideoCapture("./teste4.mp4")

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1, fy=1)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
TIMEOUT = 50  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 240  # Número de frames para considerar uma nova pessoa

# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

while True:
    ret, frame = cam.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (0, 0), fx=1, fy=1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

    # Subtrair o frame de referência do frame atual
    frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
    _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos pequenos e criar uma lista de áreas
    valid_contours = []
    areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 1000]

    if areas:
        max_area = max(areas)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area >= 1000 and area > max_area * 0.3:
                valid_contours.append(contour)

    # Loop sobre os contornos filtrados
    for contour in valid_contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        person_hist = get_color_histogram(roi)

        new_person = True
        for person in people_detected:
            dist = cv2.compareHist(person['hist'], person_hist, cv2.HISTCMP_CORREL)
            if dist > 0.8:
                person['last_seen'] = (x, y)
                person['missed_frames'] = 0
                person['detection_delay'] += 1  # Incrementa o contador de frames para essa pessoa
                new_person = False
                break

        if new_person:
            # Adiciona nova pessoa apenas após o número definido de frames
            if person_id < DETECTION_DELAY:
                person_id += 1
                people_detected.append({
                    'id': person_id,
                    'hist': person_hist,
                    'last_seen': (x, y),
                    'missed_frames': 0,
                    'detection_delay': 1  # Inicia o contador para nova detecção
                })
                cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # Se a nova pessoa ainda não tiver sido detectada por um tempo suficiente, apenas atualiza last_seen
                cv2.putText(frame, "Esperando por nova detecção...", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Desenhar a casca convexa ao redor da pessoa detectada
        hull = cv2.convexHull(contour)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

    # Atualiza o contador de frames perdidos para todas as pessoas
    for person in people_detected:
        if person['missed_frames'] > TIMEOUT:
            people_detected.remove(person)
        else:
            person['missed_frames'] += 1

    # Mostrar IDs de todas as pessoas detectadas recentemente
    for person in people_detected:
        if person['missed_frames'] <= TIMEOUT:
            cv2.putText(frame, f"Pessoa {person['id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar o frame com as detecções
    cv2.imshow('Deteccao de Mudancas', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Frame Delta', frame_delta)

    # Pressione 'q' para sair
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()


# vídeo de teste

In [10]:
import cv2
import numpy as np
import time

# Abrir o vídeo
cam = cv2.VideoCapture("./teste5.mp4")

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

fast = cv2.FastFeatureDetector_create() 
orb = cv2.ORB_create()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1, fy=1)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
false_id = 500000
TIMEOUT = 25  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 100  # Número de frames para considerar uma nova pessoa
paused = False

# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()


while True:
    if not paused:
        ret, frame = cam.read()

        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (0, 0), fx=1, fy=1)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

        # Subtrair o frame de referência do frame atual
        frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
        _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        colored_movement = cv2.bitwise_and(frame, frame, mask=thresh)

        # Encontrar contornos
        contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filtrar contornos pequenos e criar uma lista de áreas
        valid_contours = []
        areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 1000]

        if areas:
            max_area = max(areas)

            for contour in contours:
                area = cv2.contourArea(contour)
                if area >= 1000 and area > max_area * 0.3:
                    valid_contours.append(contour)

        # Loop sobre os contornos filtrados
        for contour in valid_contours:
            x, y, w, h = cv2.boundingRect(contour)
            roi = frame[y:y+h, x:x+w]
            person_hist = get_color_histogram(roi)

            new_person = True
            for person in people_detected:
                dist = cv2.compareHist(person['hist'], person_hist, cv2.HISTCMP_CORREL)
                if dist > 0.8:
                    person['last_seen'] = (x, y)
                    person['missed_frames'] = 0
                    person['detection_delay'] += 1  # Incrementa o contador de frames para essa pessoa
                    new_person = False
                    
                    # Se a pessoa já estiver detectada há mais de 3 segundos, capturar keypoints e descritores
                    if time.time() - person['time_entered'] > 3:
                        if person['false_id'] != 0: 
                            person_id += 1
                            person["real_id"] = person_id
                        person['false_id'] = 0
                        
                        hull = cv2.convexHull(contour)
                        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
                        
                        kp = fast.detect(colored_movement, None) 
                        # kp = orb.detect(colored_movement, None) 
                        kp, des = orb.compute(colored_movement, kp) 
                        person['keypoints'] = kp[:person['num_descriptors']]
                        person['descriptors'] = des[:person['num_descriptors']]
                        person['num_descriptors'] += 10  # Aumenta o número de descritores capturados

                        # Desenhar keypoints na imagem
                        # frame = cv2.drawKeypoints(frame, person['keypoints'], frame, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                        cv2.putText(frame, f"Pessoa {person['real_id']} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    break

            if new_person:
                # person_id += 1
                false_id += 1
                people_detected.append({
                    'false_id': false_id,
                    'real_id': false_id,
                    'hist': person_hist,
                    'last_seen': (x, y),
                    'missed_frames': 0,
                    'detection_delay': 1,  # Inicia o contador para nova detecção
                    'time_entered': time.time(),
                    'num_descriptors': 10,  # Inicia com 10 descritores
                    'keypoints': None,
                    'descriptors': None
                })
                # cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Desenhar a casca convexa ao redor da pessoa detectada
            # hull = cv2.convexHull(contour)
            # cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

        # Atualiza o contador de frames perdidos para todas as pessoas
        for person in people_detected:
            if person['missed_frames'] > TIMEOUT:
                people_detected.remove(person)
            else:
                person['missed_frames'] += 1

        # Mostrar IDs de todas as pessoas detectadas recentemente
        # for person in people_detected:
        #     if person['missed_frames'] <= TIMEOUT:
        #         cv2.putText(frame, f"Pessoa {person['false_id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Mostrar o frame com as detecções
        # cv2.imshow('Thresh', thresh)
        # cv2.imshow('Frame Delta', frame_delta)
        cv2.imshow('Colored', colored_movement)
        cv2.imshow('Deteccao de Mudancas', frame)

    key = cv2.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()
